In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import folium
import h3
import geopandas as gpd
import geojson

In [22]:
import sys
print(sys.path)

['C:\\Users\\marlo\\Desktop\\Master\\AAA\\AAA-AIbertus-Magnus\\tasks', 'C:\\Users\\marlo\\anaconda3\\python311.zip', 'C:\\Users\\marlo\\anaconda3\\DLLs', 'C:\\Users\\marlo\\anaconda3\\Lib', 'C:\\Users\\marlo\\anaconda3', '', 'C:\\Users\\marlo\\anaconda3\\Lib\\site-packages', 'C:\\Users\\marlo\\anaconda3\\Lib\\site-packages\\win32', 'C:\\Users\\marlo\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\marlo\\anaconda3\\Lib\\site-packages\\Pythonwin']


In [25]:
df = pd.read_csv("../data/analytics_trip_data.csv")

In [26]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])

In [27]:
df.rename(columns={"h3_res_4": "h3_res_4_pickup", "h3_res_6": "h3_res_6_pickup", "h3_res_8":"h3_res_8_pickup"},inplace=True)

In [35]:
pd.set_option('display.max_columns', None)
df.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_location,dropoff_centroid_location,h3_res_4_pickup,h3_res_6_pickup,h3_res_8_pickup,h3_res_4_dropoff,h3_res_6_dropoff,h3_res_8_dropoff,date,temperature,precipitation
0,0a0a2ca50df85134ce34f0c2c7bda6e78ca2a52c,bbe793119144e0bd97a4268af2a28ed0f3dad49545592c...,2018-01-01 00:15:00,2018-01-01 00:15:00,180.0,0.00,1.703108e+10,1.703132e+10,8.0,32.0,4.00,2.0,0.0,1.0,7.00,Credit Card,Blue Ribbon Taxi Association Inc.,POINT (-87.6262149064 41.8925077809),POINT (-87.6209929134 41.8849871918),862664c1fffffff,842664dffffffff,882664c1e1fffff,842664dffffffff,862664c1fffffff,882664c1e3fffff,2018-01-01 01:53:00,-18.333333,0.0
1,d5d00fe6f098623a12325194a3c9b9c8259addac,12052a7776cbfcf53d9daa2805ef852de2dbfb28bc38a6...,2018-01-01 00:15:00,2018-01-01 00:15:00,217.0,0.61,1.703184e+10,1.703132e+10,32.0,32.0,4.75,0.0,0.0,1.5,6.25,Cash,Taxi Affiliation Service Yellow,POINT (-87.6327464887 41.8809944707),POINT (-87.6209929134 41.8849871918),862664c1fffffff,842664dffffffff,882664c1a9fffff,842664dffffffff,862664c1fffffff,882664c1e3fffff,2018-01-01 01:53:00,-18.333333,0.0
2,4d1b86ec3bad2e8a8b0273223d79cef7600b2821,bbdec1f9949f4e13be32d6d0038763a29f09ef18534115...,2018-01-01 00:15:00,2018-01-01 00:30:00,840.0,4.20,NaN,NaN,8.0,22.0,14.50,0.0,0.0,0.0,14.50,Cash,Globe Taxi,POINT (-87.6333080367 41.899602111),POINT (-87.69915534320002 41.9227606205),862664c1fffffff,842664dffffffff,882664c1edfffff,842664dffffffff,862664ca7ffffff,882664ca11fffff,2018-01-01 01:53:00,-18.333333,0.0
3,59c63295676feae18f036452c2df04af9652f956,bda6300ce64bfe977c79eb92ade7d37b6d4e05fed81151...,2018-01-01 00:15:00,2018-01-01 00:30:00,294.0,0.85,NaN,NaN,24.0,24.0,5.00,0.0,0.0,2.0,7.00,Cash,Chicago Carriage Cab Corp,POINT (-87.6763559892 41.90120699410001),POINT (-87.6763559892 41.90120699410001),862664cafffffff,842664dffffffff,882664cac3fffff,842664dffffffff,862664cafffffff,882664cac3fffff,2018-01-01 01:53:00,-18.333333,0.0
4,12c6e02f7d25c503736dae233ce2281a815dc67a,28d27337bea07d811235348e9b25af937d110ac08ca576...,2018-01-01 00:15:00,2018-01-01 00:30:00,486.0,4.55,NaN,NaN,28.0,35.0,13.50,0.0,0.0,0.0,13.50,Cash,Chicago Carriage Cab Corp,POINT (-87.6635175498 41.874005383),POINT (-87.6186777673 41.8351179863),862664cafffffff,842664dffffffff,882664ca91fffff,842664dffffffff,862664cf7ffffff,882664cf53fffff,2018-01-01 01:53:00,-18.333333,0.0


In [39]:
#Pass dataframe as pd object and h3geocol,datacol,legend_name as strings
def create_choropleth(dataframe,h3geocol,datacol,legend_name):
    grouped_data = dataframe.groupby(dataframe[h3geocol])[datacol].mean().reset_index()
    m = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
    
    geojson_data = {
        'type': 'FeatureCollection',
        'features': []
    }
    
    for index, row in grouped_data.iterrows():
        geometry = {'type': 'Polygon', 'coordinates': [h3.h3_to_geo_boundary(row[h3geocol], geo_json=True)]}
        feature = {'type': 'Feature', 'geometry': geometry, 'properties': {f'{h3geocol}': row[h3geocol],f'{datacol}': row[datacol]}}
        geojson_data['features'].append(feature)
        
    cp = folium.Choropleth(
        geo_data=geojson_data,
        data=grouped_data,
        columns=[h3geocol, datacol],
        key_on=f'feature.properties.{h3geocol}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_name
    ).add_to(m)

    folium.GeoJsonTooltip([h3geocol,datacol]).add_to(cp.geojson)
    
    return m

In [42]:
create_choropleth(df,"h3_res_8_dropoff","trip_seconds","Average Total trip seconds")

In [41]:
create_choropleth(df,"h3_res_8_pickup","trip_total","Average total trip price")

In [ ]:
#TODO: add price per mile/km

In [229]:
census_tract_geojson =  gpd.read_file("../data/Chicago_boundaries.geojson")

In [230]:
census_tract_geojson

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,"MULTIPOLYGON (((-87.62405 41.73022, -87.62405 ..."
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,"MULTIPOLYGON (((-87.68608 41.82296, -87.68607 ..."
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,"MULTIPOLYGON (((-87.62935 41.85280, -87.62934 ..."
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,"MULTIPOLYGON (((-87.68813 41.85569, -87.68816 ..."
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,"MULTIPOLYGON (((-87.63312 41.87449, -87.63306 ..."
...,...,...,...,...,...,...,...,...,...,...
796,17,704,7,Census Tract 704,7,17031070400,,070400,031,"MULTIPOLYGON (((-87.65746 41.93258, -87.65722 ..."
797,17,705,7,Census Tract 705,7,17031070500,,070500,031,"MULTIPOLYGON (((-87.66349 41.93036, -87.66350 ..."
798,17,1303,13,Census Tract 1303,13,17031130300,,130300,031,"MULTIPOLYGON (((-87.71436 41.98300, -87.71472 ..."
799,17,2922,29,Census Tract 2922,29,17031292200,,292200,031,"MULTIPOLYGON (((-87.71317 41.85523, -87.71357 ..."
